In [1]:
import json
import sys
import os
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize
from nltk.tokenize.stanford import StanfordTokenizer
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import stopwords
from collections import defaultdict
from gensim import corpora
from nltk.tokenize import RegexpTokenizer
from gensim.models import Word2Vec
import gensim
import re
import string
import nltk
import numpy as np
import pandas as pd
#nltk.download() #use if nltk cries for any language-specific docs

In [2]:
"""ay=str()
for file in os.listdir('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10'):
    full_filename = "%s/%s" % ('Models\word2vec\English CoNLL17 corpus - gensim skipgram - 10', file)
    if full_filename.endswith('.txt'):
        with open(full_filename,'r', encoding="ISO-8859-1") as fi:
            ay=fi.readlines()
"""
subs=[]
for filename in os.listdir('Subs'):
    if filename.endswith(".json"):
        with open(os.path.join('Subs', filename)) as json_file:
            json_text = json.load(json_file)
            print(filename)
            subs.append(json_text)

OUTPUT200acres.json
OUTPUTBraincels.json
OUTPUTClownworldwar.json
OUTPUTCringeAnarchy.json
OUTPUTDarkEnlightenment.json
OUTPUTdeep_ecology.json
OUTPUTfrenworld.json
OUTPUTGenderCritical.json
OUTPUTHBD.json
OUTPUTmetacanada.json
OUTPUTMGTOW.json
OUTPUTShitPoliticsSays.json
OUTPUTTheNewRight.json
OUTPUTThe_Donald.json
OUTPUTThe_Donald2.json
OUTPUTThe_Donald3.json


In [3]:
#not used
def sub2vocab(sub):
    documents=sub2docs(sub)
    temp=[]
    stop = set(stopwords.words('english'))
    #for doc in documents:
        #print(word_tokenize(doc))
    texts =[
        [word for word in document.lower().split() if word not in stop]
        for document in documents
    ]
    frequency = defaultdict(int)
    for text in texts:
        for token in text:
            frequency[token] += 1
    texts = [
        [token for token in text if frequency[token] > 1]
        for text in texts]
    return texts

In [5]:
#not used even more
"""def sub2vocab(sub):
    raw=str()
    for i, j in sub.items():
        raw+=i
        raw+=' '
        for o,p in j.items():
            raw+=p
            raw+=' '
        raw+='\n'
    temp=[]
    stop = set(stopwords.words('english'))
    for i in word_tokenize(raw.lower()):
        if i not in stop:
            temp.append(i)
    frequency = defaultdict(int)
    for word in temp:
        frequency[word] += 1
    temp = set([token for token in temp if frequency[token] > 1])
    return temp"""

"def sub2vocab(sub):\n    raw=str()\n    for i, j in sub.items():\n        raw+=i\n        raw+=' '\n        for o,p in j.items():\n            raw+=p\n            raw+=' '\n        raw+='\n'\n    temp=[]\n    stop = set(stopwords.words('english'))\n    for i in word_tokenize(raw.lower()):\n        if i not in stop:\n            temp.append(i)\n    frequency = defaultdict(int)\n    for word in temp:\n        frequency[word] += 1\n    temp = set([token for token in temp if frequency[token] > 1])\n    return temp"

In [6]:
def sub2docs(sub):
    documents=[]
    trans_table = str.maketrans("", "", string.punctuation+"’”")
    for i, j in sub.items():
        temp=re.sub(r'\d+','',i.lower().translate(trans_table)).replace('\n', '')
        documents.append(temp)
        for o,p in j.items():
            temp=re.sub(r'\d+', '',p.lower().translate(trans_table)).replace('\n', '')
            documents.append(temp)
    frequency = defaultdict(int)
    for document in documents:
        for token in document.split():
            frequency[token] += 1
    documents = [
        [token for token in document.split() if frequency[token] > 1]
        for document in documents]
    return documents

In [7]:
"""def removePunctuation(words):
    trans_table = str.maketrans("", "", string.punctuation)
    return set([s.translate(trans_table) for s in words])"""

'def removePunctuation(words):\n    trans_table = str.maketrans("", "", string.punctuation)\n    return set([s.translate(trans_table) for s in words])'

In [8]:
def target_wv2distances(wv,missing):
    dists=np.zeros((len(wv.vocab),len(wv.vocab)))
    words=[]
    for word in wv.vocab.keys():
        words.append(word)
    for i in range(len(words)):
        if words[i] not in missing:
            print(i,len(words))
            for j in range(len(words)):
                if words[j] not in missing:
                    j+=i
                    if j<len(words):
                        if words[j] not in missing:
                            dists[i,j]=wv.n_similarity([words[i]],[words[j]])
    return dists

In [9]:
def ref_wv2distances(target_wv,reference_wv):
    dists=np.zeros((len(target_wv.vocab),len(target_wv.vocab)))
    words=[]
    for word in target_wv.vocab.keys():
        words.append(word)
    start=0
    missing=[]
    for i in range(len(words)):
        print(i,len(words))
        for j in range(len(words)):
            j+=start
            try:
                if j<len(words):
                    dists[i,j]=reference_wv.n_similarity([words[i]],[words[j]])
            except Exception as e:
                word=re.findall('''"word '(.+?)' not in vocabulary''', str(e))[0]
                if word not in missing:
                    #pass
                    print("type error: " + str(e))
                    missing.append(word)
                else:
                    pass
        start+=1
    return dists, missing


In [10]:
def docs2wv(docs,name='model'):
    model = Word2Vec(docs, size=300, window=5, workers=3, sg=1, min_count=1, negative=8)
    if name!=None:
        model.save(name+".model")
    word_vectors = model.wv
    frequency = defaultdict(int)
    for document in docs:
        for token in document:
            frequency[token] += 1
    return word_vectors, frequency

In [11]:
def remove_diagonal(dists):
    for i in range(dists.shape[0]):
        for j in range(dists.shape[1]):
            if i==j:
                dists[i,j]=0
    return dists

In [12]:
docs=[]
for sub in subs:
    doc=sub2docs(sub)
    docs.extend(doc)
docs[0]

['welcome', 'announcement']

In [14]:
target_wv,frequency=docs2wv(docs,name='target')
ref_wv=gensim.models.KeyedVectors.load_word2vec_format(os.path.join(os.path.dirname('Models\word2vec\English_CoNLL17\ '), 'model.bin'), binary=True).wv

E:\Anaconda33\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [15]:
dists_ref,missing=ref_wv2distances(target_wv,ref_wv)
dists_t=target_wv2distances(target_wv,missing)
dists_ref = remove_diagonal(dists_ref)
dists_t  = remove_diagonal(dists_t)

0 28188
type error: "word 'chadright' not in vocabulary"
type error: "word 'shoahd' not in vocabulary"
type error: "word 'thuleanperspective' not in vocabulary"
type error: "word 'plebbit' not in vocabulary"
type error: "word 'memelords' not in vocabulary"
type error: "word 'riotim' not in vocabulary"
type error: "word 'pinesim' not in vocabulary"
type error: "word 'lolberts' not in vocabulary"
type error: "word 'racres' not in vocabulary"
type error: "word 'fashwave' not in vocabulary"
type error: "word 'antiwomen' not in vocabulary"
type error: "word 'altright' not in vocabulary"
type error: "word 'judeochristians' not in vocabulary"
type error: "word 'igtt' not in vocabulary"
type error: "word '“the' not in vocabulary"
type error: "word 'rstrangestatistics' not in vocabulary"
type error: "word 'rrakers' not in vocabulary"
type error: "word 'elsagate' not in vocabulary"
type error: "word 'selfdestructive' not in vocabulary"
type error: "word 'racemixing' not in vocabulary"
type error

type error: "word 'dickcels' not in vocabulary"
type error: "word 'numale' not in vocabulary"
type error: "word 'spoilerpilled' not in vocabulary"
type error: "word 'braziliansmegma' not in vocabulary"
type error: "word 'normalfag' not in vocabulary"
type error: "word 'retardcel' not in vocabulary"
type error: "word 'soycuck' not in vocabulary"
type error: "word 'inquells' not in vocabulary"
type error: "word 'nearcel' not in vocabulary"
type error: "word 'surgerymaxxing' not in vocabulary"
type error: "word 'dickcel' not in vocabulary"
type error: "word 'rtall' not in vocabulary"
type error: "word '😤' not in vocabulary"
type error: "word 'brojust' not in vocabulary"
type error: "word 'talentcel' not in vocabulary"
type error: "word 'brainlets' not in vocabulary"
type error: "word 'itfags' not in vocabulary"
type error: "word 'nonvirgin' not in vocabulary"
type error: "word 'statusmaxxed' not in vocabulary"
type error: "word 'drnerdlove' not in vocabulary"
type error: "word 'honklers' 

type error: "word 'passivenihilism' not in vocabulary"
type error: "word 'activenihilism' not in vocabulary"
type error: "word 'activenihilist' not in vocabulary"
type error: "word '“in' not in vocabulary"
type error: "word 'ethnat' not in vocabulary"
type error: "word 'ethnats' not in vocabulary"
type error: "word 'evolas' not in vocabulary"
type error: "word 'neoreactionaries' not in vocabulary"
type error: "word 'revidpedia' not in vocabulary"
type error: "word 'bioleninism' not in vocabulary"
type error: "word 'clowardpiven' not in vocabulary"
type error: "word 'stateinexile' not in vocabulary"
type error: "word 'accelerationists' not in vocabulary"
type error: "word '“rivers' not in vocabulary"
type error: "word 'itxb' not in vocabulary"
type error: "word 'noninvading' not in vocabulary"
type error: "word 'atavisionary' not in vocabulary"
type error: "word 'cuckery' not in vocabulary"
type error: "word 'satact' not in vocabulary"
type error: "word 'moldbugs' not in vocabulary"
typ

type error: "word 'nullfud' not in vocabulary"
type error: "word 'nananananananana' not in vocabulary"
type error: "word 'dothats' not in vocabulary"
type error: "word 'repeaking' not in vocabulary"
type error: "word '“daughter' not in vocabulary"
type error: "word '‘woke' not in vocabulary"
type error: "word 'rtransgender' not in vocabulary"
type error: "word 'transrights' not in vocabulary"
type error: "word 'xbhe' not in vocabulary"
type error: "word 'selfid' not in vocabulary"
type error: "word 'xbmy' not in vocabulary"
type error: "word 'transtrenders' not in vocabulary"
type error: "word 'singlesex' not in vocabulary"
type error: "word 'selfabsorbed' not in vocabulary"
type error: "word 'libfem' not in vocabulary"
type error: "word 'hyperfeminine' not in vocabulary"
type error: "word 'hatereading' not in vocabulary"
type error: "word 'transidentified' not in vocabulary"
type error: "word 'terfi' not in vocabulary"
type error: "word 'preeverything' not in vocabulary"
type error: "

type error: "word 'sexfocused' not in vocabulary"
type error: "word 'selfsacrifice' not in vocabulary"
type error: "word 'femalesimply' not in vocabulary"
type error: "word 'contextfeminine' not in vocabulary"
type error: "word 'stupidalso' not in vocabulary"
type error: "word '“taking' not in vocabulary"
type error: "word 'yanivs' not in vocabulary"
type error: "word '“thank' not in vocabulary"
type error: "word '“she' not in vocabulary"
type error: "word '“wtf' not in vocabulary"
type error: "word '“was' not in vocabulary"
type error: "word '“sexist' not in vocabulary"
type error: "word 'gendertrender' not in vocabulary"
type error: "word 'antitrans' not in vocabulary"
type error: "word 'terves' not in vocabulary"
type error: "word 'wellunderstood' not in vocabulary"
type error: "word 'propedophilia' not in vocabulary"
type error: "word 'tooim' not in vocabulary"
type error: "word 'ranarchism' not in vocabulary"
type error: "word 'rgendercritical' not in vocabulary"
type error: "word

...

24529 28188
24530 28188
24531 28188
24532 28188
24534 28188
24535 28188
24537 28188
24538 28188
24539 28188
24540 28188
24541 28188
24542 28188
24543 28188
24544 28188
24545 28188
24546 28188
24547 28188
24548 28188
24549 28188
24550 28188
24551 28188
24555 28188
24556 28188
24557 28188
24558 28188
24559 28188
24560 28188
24561 28188
24562 28188
24563 28188
24564 28188
24565 28188
24567 28188
24568 28188
24569 28188
24570 28188
24571 28188
24574 28188
24575 28188
24576 28188
24578 28188
24579 28188
24580 28188
24581 28188
24583 28188
24584 28188
24585 28188
24586 28188
24591 28188
24592 28188
24593 28188
24594 28188
24595 28188
24596 28188
24597 28188
24598 28188
24599 28188
24600 28188
24601 28188
24602 28188
24603 28188
24604 28188
24605 28188
24606 28188
24607 28188
24609 28188
24610 28188
24611 28188
24612 28188
24613 28188
24614 28188
24615 28188
24616 28188
24617 28188
24618 28188
24619 28188
24620 28188
24621 28188
24622 28188
24623 28188
24624 28188
24625 28188
24626 28188
2462

25363 28188
25364 28188
25365 28188
25366 28188
25367 28188
25370 28188
25371 28188
25373 28188
25374 28188
25375 28188
25377 28188
25379 28188
25381 28188
25382 28188
25385 28188
25386 28188
25387 28188
25388 28188
25389 28188
25391 28188
25392 28188
25394 28188
25395 28188
25396 28188
25397 28188
25399 28188
25401 28188
25402 28188
25403 28188
25404 28188
25405 28188
25406 28188
25408 28188
25410 28188
25411 28188
25412 28188
25413 28188
25414 28188
25416 28188
25417 28188
25419 28188
25420 28188
25421 28188
25424 28188
25425 28188
25426 28188
25427 28188
25428 28188
25430 28188
25432 28188
25433 28188
25434 28188
25435 28188
25437 28188
25438 28188
25439 28188
25441 28188
25442 28188
25443 28188
25444 28188
25445 28188
25447 28188
25449 28188
25450 28188
25451 28188
25452 28188
25453 28188
25454 28188
25455 28188
25458 28188
25459 28188
25460 28188
25462 28188
25463 28188
25464 28188
25465 28188
25466 28188
25467 28188
25468 28188
25469 28188
25470 28188
25471 28188
25473 28188
2547

26185 28188
26186 28188
26187 28188
26188 28188
26190 28188
26191 28188
26192 28188
26193 28188
26194 28188
26196 28188
26197 28188
26198 28188
26199 28188
26200 28188
26201 28188
26202 28188
26203 28188
26204 28188
26205 28188
26206 28188
26207 28188
26208 28188
26209 28188
26210 28188
26211 28188
26212 28188
26213 28188
26214 28188
26215 28188
26216 28188
26217 28188
26218 28188
26219 28188
26222 28188
26224 28188
26225 28188
26226 28188
26227 28188
26229 28188
26230 28188
26231 28188
26232 28188
26233 28188
26234 28188
26235 28188
26236 28188
26237 28188
26238 28188
26239 28188
26240 28188
26241 28188
26242 28188
26243 28188
26244 28188
26246 28188
26247 28188
26248 28188
26249 28188
26250 28188
26252 28188
26253 28188
26254 28188
26255 28188
26256 28188
26257 28188
26258 28188
26259 28188
26260 28188
26261 28188
26262 28188
26263 28188
26264 28188
26265 28188
26266 28188
26267 28188
26268 28188
26269 28188
26270 28188
26271 28188
26272 28188
26273 28188
26274 28188
26275 28188
2627

27006 28188
27007 28188
27008 28188
27009 28188
27010 28188
27011 28188
27012 28188
27013 28188
27014 28188
27015 28188
27016 28188
27017 28188
27019 28188
27021 28188
27022 28188
27023 28188
27024 28188
27025 28188
27026 28188
27028 28188
27029 28188
27030 28188
27031 28188
27032 28188
27033 28188
27034 28188
27035 28188
27036 28188
27037 28188
27038 28188
27039 28188
27040 28188
27041 28188
27042 28188
27043 28188
27045 28188
27046 28188
27047 28188
27048 28188
27049 28188
27050 28188
27051 28188
27052 28188
27053 28188
27054 28188
27055 28188
27056 28188
27057 28188
27058 28188
27059 28188
27061 28188
27062 28188
27063 28188
27064 28188
27065 28188
27066 28188
27067 28188
27068 28188
27069 28188
27070 28188
27071 28188
27072 28188
27073 28188
27074 28188
27075 28188
27076 28188
27077 28188
27078 28188
27079 28188
27080 28188
27081 28188
27082 28188
27083 28188
27084 28188
27085 28188
27086 28188
27088 28188
27089 28188
27090 28188
27091 28188
27092 28188
27093 28188
27095 28188
2709

27819 28188
27820 28188
27821 28188
27822 28188
27823 28188
27824 28188
27825 28188
27826 28188
27827 28188
27828 28188
27829 28188
27830 28188
27831 28188
27832 28188
27833 28188
27835 28188
27836 28188
27837 28188
27838 28188
27840 28188
27841 28188
27842 28188
27843 28188
27846 28188
27847 28188
27848 28188
27849 28188
27850 28188
27851 28188
27852 28188
27853 28188
27854 28188
27856 28188
27857 28188
27858 28188
27859 28188
27860 28188
27861 28188
27862 28188
27863 28188
27864 28188
27865 28188
27866 28188
27867 28188
27868 28188
27870 28188
27871 28188
27873 28188
27875 28188
27877 28188
27880 28188
27881 28188
27882 28188
27883 28188
27885 28188
27886 28188
27887 28188
27888 28188
27889 28188
27890 28188
27891 28188
27892 28188
27893 28188
27894 28188
27895 28188
27896 28188
27897 28188
27898 28188
27900 28188
27901 28188
27902 28188
27903 28188
27904 28188
27905 28188
27906 28188
27907 28188
27908 28188
27909 28188
27910 28188
27912 28188
27913 28188
27914 28188
27915 28188
2791

In [16]:
"""e='''type error: "word 'memelords' not in vocabulary"'''
word=re.findall(''' error: "word '(.+?)' not in vocabulary''', str(e))[0]
print(word)"""

'e=\'\'\'type error: "word \'memelords\' not in vocabulary"\'\'\'\nword=re.findall(\'\'\' error: "word \'(.+?)\' not in vocabulary\'\'\', str(e))[0]\nprint(word)'

In [17]:
ranks=pd.DataFrame(columns=['Value','Count','Prominence'])
dists_comp=np.absolute(dists_t-dists_ref)

In [18]:
i=0
#dists_comp=np.absolute(dists_t-dists_ref)
for word in target_wv.vocab.keys():
    print(i)
    temp=np.sum(dists_comp[i,:])+np.sum(dists_comp[:,i])
    ranks.loc[word]=[temp,frequency[word],0]
    i+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100


3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732
3733
3734
3735
3736
3737
3738
3739
3740
3741
3742
3743
3744


5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370
5371
5372
5373
5374
5375
5376
5377
5378
5379
5380
5381
5382
5383
5384
5385
5386
5387
5388
5389
5390


6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017
7018
7019
7020
7021
7022
7023
7024
7025
7026
7027
7028
7029


...

22509
22510
22511
22512
22513
22514
22515
22516
22517
22518
22519
22520
22521
22522
22523
22524
22525
22526
22527
22528
22529
22530
22531
22532
22533
22534
22535
22536
22537
22538
22539
22540
22541
22542
22543
22544
22545
22546
22547
22548
22549
22550
22551
22552
22553
22554
22555
22556
22557
22558
22559
22560
22561
22562
22563
22564
22565
22566
22567
22568
22569
22570
22571
22572
22573
22574
22575
22576
22577
22578
22579
22580
22581
22582
22583
22584
22585
22586
22587
22588
22589
22590
22591
22592
22593
22594
22595
22596
22597
22598
22599
22600
22601
22602
22603
22604
22605
22606
22607
22608
22609
22610
22611
22612
22613
22614
22615
22616
22617
22618
22619
22620
22621
22622
22623
22624
22625
22626
22627
22628
22629
22630
22631
22632
22633
22634
22635
22636
22637
22638
22639
22640
22641
22642
22643
22644
22645
22646
22647
22648
22649
22650
22651
22652
22653
22654
22655
22656
22657
22658
22659
22660
22661
22662
22663
22664
22665
22666
22667
22668
22669
22670
22671
22672
22673
22674
2267

23880
23881
23882
23883
23884
23885
23886
23887
23888
23889
23890
23891
23892
23893
23894
23895
23896
23897
23898
23899
23900
23901
23902
23903
23904
23905
23906
23907
23908
23909
23910
23911
23912
23913
23914
23915
23916
23917
23918
23919
23920
23921
23922
23923
23924
23925
23926
23927
23928
23929
23930
23931
23932
23933
23934
23935
23936
23937
23938
23939
23940
23941
23942
23943
23944
23945
23946
23947
23948
23949
23950
23951
23952
23953
23954
23955
23956
23957
23958
23959
23960
23961
23962
23963
23964
23965
23966
23967
23968
23969
23970
23971
23972
23973
23974
23975
23976
23977
23978
23979
23980
23981
23982
23983
23984
23985
23986
23987
23988
23989
23990
23991
23992
23993
23994
23995
23996
23997
23998
23999
24000
24001
24002
24003
24004
24005
24006
24007
24008
24009
24010
24011
24012
24013
24014
24015
24016
24017
24018
24019
24020
24021
24022
24023
24024
24025
24026
24027
24028
24029
24030
24031
24032
24033
24034
24035
24036
24037
24038
24039
24040
24041
24042
24043
24044
24045
2404

25251
25252
25253
25254
25255
25256
25257
25258
25259
25260
25261
25262
25263
25264
25265
25266
25267
25268
25269
25270
25271
25272
25273
25274
25275
25276
25277
25278
25279
25280
25281
25282
25283
25284
25285
25286
25287
25288
25289
25290
25291
25292
25293
25294
25295
25296
25297
25298
25299
25300
25301
25302
25303
25304
25305
25306
25307
25308
25309
25310
25311
25312
25313
25314
25315
25316
25317
25318
25319
25320
25321
25322
25323
25324
25325
25326
25327
25328
25329
25330
25331
25332
25333
25334
25335
25336
25337
25338
25339
25340
25341
25342
25343
25344
25345
25346
25347
25348
25349
25350
25351
25352
25353
25354
25355
25356
25357
25358
25359
25360
25361
25362
25363
25364
25365
25366
25367
25368
25369
25370
25371
25372
25373
25374
25375
25376
25377
25378
25379
25380
25381
25382
25383
25384
25385
25386
25387
25388
25389
25390
25391
25392
25393
25394
25395
25396
25397
25398
25399
25400
25401
25402
25403
25404
25405
25406
25407
25408
25409
25410
25411
25412
25413
25414
25415
25416
2541

26621
26622
26623
26624
26625
26626
26627
26628
26629
26630
26631
26632
26633
26634
26635
26636
26637
26638
26639
26640
26641
26642
26643
26644
26645
26646
26647
26648
26649
26650
26651
26652
26653
26654
26655
26656
26657
26658
26659
26660
26661
26662
26663
26664
26665
26666
26667
26668
26669
26670
26671
26672
26673
26674
26675
26676
26677
26678
26679
26680
26681
26682
26683
26684
26685
26686
26687
26688
26689
26690
26691
26692
26693
26694
26695
26696
26697
26698
26699
26700
26701
26702
26703
26704
26705
26706
26707
26708
26709
26710
26711
26712
26713
26714
26715
26716
26717
26718
26719
26720
26721
26722
26723
26724
26725
26726
26727
26728
26729
26730
26731
26732
26733
26734
26735
26736
26737
26738
26739
26740
26741
26742
26743
26744
26745
26746
26747
26748
26749
26750
26751
26752
26753
26754
26755
26756
26757
26758
26759
26760
26761
26762
26763
26764
26765
26766
26767
26768
26769
26770
26771
26772
26773
26774
26775
26776
26777
26778
26779
26780
26781
26782
26783
26784
26785
26786
2678

27987
27988
27989
27990
27991
27992
27993
27994
27995
27996
27997
27998
27999
28000
28001
28002
28003
28004
28005
28006
28007
28008
28009
28010
28011
28012
28013
28014
28015
28016
28017
28018
28019
28020
28021
28022
28023
28024
28025
28026
28027
28028
28029
28030
28031
28032
28033
28034
28035
28036
28037
28038
28039
28040
28041
28042
28043
28044
28045
28046
28047
28048
28049
28050
28051
28052
28053
28054
28055
28056
28057
28058
28059
28060
28061
28062
28063
28064
28065
28066
28067
28068
28069
28070
28071
28072
28073
28074
28075
28076
28077
28078
28079
28080
28081
28082
28083
28084
28085
28086
28087
28088
28089
28090
28091
28092
28093
28094
28095
28096
28097
28098
28099
28100
28101
28102
28103
28104
28105
28106
28107
28108
28109
28110
28111
28112
28113
28114
28115
28116
28117
28118
28119
28120
28121
28122
28123
28124
28125
28126
28127
28128
28129
28130
28131
28132
28133
28134
28135
28136
28137
28138
28139
28140
28141
28142
28143
28144
28145
28146
28147
28148
28149
28150
28151
28152
2815

In [19]:
ranks['Value']=ranks['Value']/ranks['Value'].mean()

In [20]:
ranks['Prominence']=ranks['Value']*np.log(ranks['Count'])

In [21]:
ranks=ranks.sort_values('Value',ascending=False)

In [22]:
ranks.to_csv('result.csv')

In [23]:
ranks=pd.read_csv('result.csv')

In [24]:
ranks.to_csv('result.csv')

In [25]:
dists_t.min()

-0.2068127732949537

In [26]:
dists_ref.min()

-0.2479589724469528

In [33]:
filtered=ranks[ranks['Count']>10]

In [ ]:
filtered

In [29]:
filtered.columns

Index(['Unnamed: 0', 'Value', 'Count', 'Prominence'], dtype='object')

In [30]:
ref_wv=gensim.models.KeyedVectors.load_word2vec_format(
    os.path.join(os.path.dirname('Models\word2vec\English_CoNLL17\ '), 'model.bin'), binary=True).wv
words=[]
for key in ref_wv.vocab.keys():
    words.append(key)

E:\Anaconda33\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


In [71]:
typos=[]
correct=[]
name='pietruczuk'
for i in range(len(name)):
    try:
        temp0=name[i+1]+name[i]
        temp1=name[i]+name[i+1]
        typos.append(temp0)
        correct.append(temp1)
    except:
        pass
    try:
        temp0=name[i+1]+name[i]+name[i+2]
        temp1=name[i]+name[i+1]+name[i+2]
        typos.append(temp0)
        correct.append(temp1)
    except:
        pass
    if i-1>=0:
        try:
            temp0=name[i-1]+name[i+1]+name[i]
            temp1=name[i-1]+name[i]+name[i+1]
            typos.append(temp0)
            correct.append(temp1)
        except:
            pass
        try:
            temp0=name[i-1]+name[i+1]+name[i]+name[i+2]
            temp1=name[i-1]+name[i]+name[i+1]+name[i+2]
            typos.append(temp0)
            correct.append(temp1)
        except:
            pass

In [72]:
typo_df=pd.DataFrame(np.zeros((1,len(typos))),columns=typos)
corr_df=pd.DataFrame(np.zeros((1,len(correct))),columns=correct)
corr_df

,pi,pie,ie,iet,pie,piet,et,etr,iet,ietr,...,cz,czu,ucz,uczu,zu,zuk,czu,czuk,uk,zuk
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [73]:
i=0
for word in words:
    if i%10000==0:
        print(i,len(words))
    for piece in typo_df.columns:
        if piece in word:
            typo_df[piece]+=1
    for piece in corr_df.columns:
        if piece in word:
            corr_df[piece]+=1
    i+=1
    if i%500000==0:
        print(typo_df)
        print(corr_df)

0 4027169
10000 4027169
20000 4027169
30000 4027169
40000 4027169
50000 4027169
60000 4027169
70000 4027169
80000 4027169
90000 4027169
100000 4027169
110000 4027169
120000 4027169
130000 4027169
140000 4027169
150000 4027169
160000 4027169
170000 4027169
180000 4027169
190000 4027169
200000 4027169
210000 4027169
220000 4027169
230000 4027169
240000 4027169
250000 4027169
260000 4027169
270000 4027169
280000 4027169
290000 4027169
300000 4027169
310000 4027169
320000 4027169
330000 4027169
340000 4027169
350000 4027169
360000 4027169
370000 4027169
380000 4027169
390000 4027169
400000 4027169
410000 4027169
420000 4027169
430000 4027169
440000 4027169
450000 4027169
460000 4027169
470000 4027169
480000 4027169
490000 4027169
       ip    ipe      ei    eit    pei  peit       te     ter     ite   iter  \
0  4665.0  405.0  6001.0  466.0  105.0   4.0  31595.0  9410.0  2947.0  365.0   

  ...      zc  zcu  uzc  uzcu     uz  uzk   cuz  cuzk      ku  zku  
0 ...   104.0  6.0  4.0   0.0  747

2970000 4027169
2980000 4027169
2990000 4027169
        ip     ipe       ei     eit    pei  peit        te      ter      ite  \
0  30629.0  2789.0  38048.0  3155.0  728.0  57.0  195778.0  56249.0  19948.0   

     iter  ...      zc   zcu   uzc  uzcu      uz   uzk    cuz  cuzk       ku  \
0  2167.0  ...   924.0  50.0  28.0   3.0  5096.0  25.0  246.0   0.0  16254.0   

    zku  
0  37.0  

[1 rows x 32 columns]
        pi     pie       ie     iet     pie   piet        et      etr     iet  \
0  41476.0  6506.0  83611.0  5802.0  6506.0  251.0  103386.0  12796.0  5802.0   

    ietr  ...        cz    czu    ucz  uczu      zu    zuk    czu  czuk  \
0  181.0  ...    3915.0  228.0  214.0   0.0  4456.0  768.0  228.0  52.0   

        uk    zuk  
0  12467.0  768.0  

[1 rows x 32 columns]
3000000 4027169
3010000 4027169
3020000 4027169
3030000 4027169
3040000 4027169
3050000 4027169
3060000 4027169
3070000 4027169
3080000 4027169
3090000 4027169
3100000 4027169
3110000 4027169
3120000 4027169
31

In [74]:
rat=typo_df.values/corr_df.values
print(typo_df,'\n',corr_df)

        ip     ipe       ei     eit    pei  peit        te      ter      ite  \
0  42322.0  3863.0  50816.0  4260.0  966.0  78.0  270821.0  78157.0  27782.0   

     iter  ...       zc   zcu   uzc  uzcu      uz   uzk    cuz  cuzk       ku  \
0  2938.0  ...   1266.0  69.0  36.0   6.0  6801.0  32.0  344.0   0.0  21692.0   

    zku  
0  49.0  

[1 rows x 32 columns] 
         pi     pie        ie     iet     pie   piet        et      etr  \
0  56218.0  8704.0  114295.0  7934.0  8704.0  331.0  143410.0  17512.0   

      iet   ietr   ...        cz    czu    ucz  uczu      zu     zuk    czu  \
0  7934.0  238.0   ...    5404.0  314.0  298.0   2.0  5857.0  1054.0  314.0   

   czuk       uk     zuk  
0  70.0  16851.0  1054.0  

[1 rows x 32 columns]


In [75]:
ratios=[]
for i in range(len(typos)):
    ratios.append(typos[i]+':'+correct[i])
ratios

['ip:pi',
 'ipe:pie',
 'ei:ie',
 'eit:iet',
 'pei:pie',
 'peit:piet',
 'te:et',
 'ter:etr',
 'ite:iet',
 'iter:ietr',
 'rt:tr',
 'rtu:tru',
 'ert:etr',
 'ertu:etru',
 'ur:ru',
 'urc:ruc',
 'tur:tru',
 'turc:truc',
 'cu:uc',
 'cuz:ucz',
 'rcu:ruc',
 'rcuz:rucz',
 'zc:cz',
 'zcu:czu',
 'uzc:ucz',
 'uzcu:uczu',
 'uz:zu',
 'uzk:zuk',
 'cuz:czu',
 'cuzk:czuk',
 'ku:uk',
 'zku:zuk']

In [76]:
df=pd.DataFrame(rat,ratios)
df

ValueError: Shape of passed values is (32, 1), indices imply (32, 32)

In [ ]:
with open("Missing.txt", "w",encoding='utf-8') as text_file:
    print(missing, file=text_file)

In [ ]:
#vocab=[]
#for key in ref_wv.vocab.keys():
#    vocab.append(key)
#with open("Vocab.txt", "w",encoding='utf-8') as text_file:
#    print(vocab, file=text_file)

In [ ]:
filtered.to_csv('filtered.csv')

['im:mi',
 'img:mig',
 'gi:ig',
 'gid:igd',
 'mgi:mig',
 'mgid:migd',
 'dg:gd',
 'dga:gda',
 'idg:igd',
 'idga:igda',
 'ad:da',
 'adl:dal',
 'gad:gda',
 'gadl:gdal',
 'la:al',
 'dla:dal']

,0
im:mi,0.658700
img:mig,0.243649
gi:ig,0.807649
gid:igd,1.605769
mgi:mig,0.060624
mgid:migd,0.142857
dg:gd,2.229558
dga:gda,0.373424
idg:igd,6.038462
idga:igda,0.800000


In [42]:
word='midgal'
for piece in typo_df.columns:
    if piece in word:
        #print(piece,word)
        typo_df[piece].iloc[0]+=1
        print(typo_df)

    im  img   gi  gid  mgi  mgid   dg  dga  idg  idga   ad  adl  gad  gadl  \
0  0.0  0.0  0.0  0.0  0.0   0.0  4.0  3.0  3.0   3.0  0.0  0.0  0.0   0.0   

    la  dla  
0  0.0  0.0  
    im  img   gi  gid  mgi  mgid   dg  dga  idg  idga   ad  adl  gad  gadl  \
0  0.0  0.0  0.0  0.0  0.0   0.0  4.0  4.0  3.0   3.0  0.0  0.0  0.0   0.0   

    la  dla  
0  0.0  0.0  
    im  img   gi  gid  mgi  mgid   dg  dga  idg  idga   ad  adl  gad  gadl  \
0  0.0  0.0  0.0  0.0  0.0   0.0  4.0  4.0  4.0   3.0  0.0  0.0  0.0   0.0   

    la  dla  
0  0.0  0.0  
    im  img   gi  gid  mgi  mgid   dg  dga  idg  idga   ad  adl  gad  gadl  \
0  0.0  0.0  0.0  0.0  0.0   0.0  4.0  4.0  4.0   4.0  0.0  0.0  0.0   0.0   

    la  dla  
0  0.0  0.0  
